In [ ]:
# Google Colabでkaggle.jsonをアップロード
from google.colab import files
files.upload()

In [ ]:
# Google Colabでkaggle.jsonをフォルダ移動
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Google Colabでデータセットをダウンロード
!pip install kaggle
!kaggle competitions download -c flower-classification-with-tpus

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#ignore warnings
import warnings
warnings.filterwarnings('ignore')